In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

import numpy as np
import pandas as pd

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from wordcloud import WordCloud
from spellchecker import SpellChecker
#!pip install pyspellchecker

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5)
%matplotlib inline

print("Tensorflow Version",tf.__version__)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'spellchecker'

!pip install pyspellchecker

In [3]:
df=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


### EDA
* Target percent
* Keyword per count
* Text length per Target percent

In [4]:

f,ax = plt.subplots(1,2,figsize=(15,6))
df['target'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot - target')
sns.countplot('target', data=df, ax=ax[1])
ax[1].set_title('Count plot - target')
plt.show()
#target == 0 : negative(fake)
#tarege == 1 : postive(real)

NameError: name 'plt' is not defined

In [5]:

keyword_unique = df['keyword'].unique()
keyword_unique


array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [6]:

f,ax = plt.subplots(1,1,figsize=(100,10))
sns.countplot('keyword',data=df,ax=ax,order=df['keyword'].value_counts().index)
ax.set_title('Count plot - keyword')
ax.set_xlim(0,30)

plt.show()


NameError: name 'plt' is not defined

In [7]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
def get_length(text):
    return len(text)

In [9]:
df['length'] = df['text'].apply(get_length)

In [10]:
df

,id,keyword,location,text,target,length
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,83
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,125
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,137


In [11]:
plt.figure(figsize=(18,6))
plt.hist(df[df['target'] == 0]['length'], alpha = 0.6, bins=150, label = 'Not')
plt.hist(df[df['target'] == 1]['length'], alpha = 0.8, bins=150, label = 'Real')
plt.xlabel('length')
plt.ylabel('numbers')
plt.legend(loc='upper right')
plt.xlim(0,150)
plt.grid()
plt.show()

NameError: name 'plt' is not defined

### Preprocessing

* Nomalization

In [12]:
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

In [13]:
def preprocess(text,stem=False):
    text = text.lower()  # lowercase

    text = re.sub(r'[!]+', '!', text)
    text = re.sub(r'[?]+', '?', text)
    text = re.sub(r'[.]+', '.', text)
    text = re.sub(r"'", "", text)
    text = re.sub('\s+', ' ', text).strip()  # Remove and double spaces
    text = re.sub(r'&amp;?', r'and', text)  # replace & -> and
    text = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", text)  # Remove URLs
    # remove some puncts (except . ! # ?)
    text = re.sub(r'[:"$%&\*+,-/:;<=>@\\^_`{|}~]+', '', text)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'EMOJI', text)
    
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [14]:
spell = SpellChecker()
def correct_spell(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

NameError: name 'SpellChecker' is not defined

In [15]:
text = "corect"
correct_spell(text)

NameError: name 'correct_spell' is not defined

In [16]:
df.text = df.text.apply(lambda x: preprocess(x))
#df.text = df.text.apply(lambda x: correct_spell(x))

In [17]:
df.head(10)

,id,keyword,location,text,target,length
0,1,NaN,NaN,deeds reason #earthquake may allah forgive us,1,69
1,4,NaN,NaN,forest fire near la ronge sask canada,1,38
2,5,NaN,NaN,residents asked shelter place notified officer...,1,133
3,6,NaN,NaN,13000 people receive #wildfires evacuation ord...,1,65
4,7,NaN,NaN,got sent photo ruby #alaska smoke #wildfires p...,1,88
5,8,NaN,NaN,#rockyfire update california hwy 20 closed dir...,1,110
6,10,NaN,NaN,#flood #disaster heavy rain causes flash flood...,1,95
7,13,NaN,NaN,im top hill see fire woods,1,59
8,14,NaN,NaN,theres emergency evacuation happening building...,1,79
9,15,NaN,NaN,im afraid tornado coming area,1,52


* Real Words

In [18]:
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000, width = 1600, height = 800).generate(" ".join(df[df.target == 1].text))
plt.axis('off')
plt.imshow(wc,interpolation = 'bilinear')

NameError: name 'plt' is not defined

* Fake words

In [19]:
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000, width = 1600, height = 800).generate(" ".join(df[df.target == 0].text))
plt.axis('off')
plt.imshow(wc,interpolation = 'bilinear')

NameError: name 'plt' is not defined

In [20]:
df.drop(['keyword','id','location','length'],axis=1,inplace=True)
df.head(10)

,text,target
0,deeds reason #earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,residents asked shelter place notified officer...,1
3,13000 people receive #wildfires evacuation ord...,1
4,got sent photo ruby #alaska smoke #wildfires p...,1
5,#rockyfire update california hwy 20 closed dir...,1
6,#flood #disaster heavy rain causes flash flood...,1
7,im top hill see fire woods,1
8,theres emergency evacuation happening building...,1
9,im afraid tornado coming area,1


In [21]:
train_data, test_data = train_test_split(df,test_size = 0.2)
print("Train Data size : ",len(train_data))
print("Test Data size : ",len(test_data))

NameError: name 'train_test_split' is not defined

In [22]:
train_data.head(10)

NameError: name 'train_data' is not defined

In [23]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.text)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
print(word_index)
print("Vocab size : ",vocab_size)

NameError: name 'train_data' is not defined

In [24]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.text), maxlen = 30)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.text), maxlen = 30)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

NameError: name 'train_data' is not defined

In [25]:
labels = train_data.target.unique().tolist()
labels

NameError: name 'train_data' is not defined

In [26]:
y_train = train_data.target
y_test = test_data.target

print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print(y_train,y_test)

NameError: name 'train_data' is not defined

### Word Embedding
* !wget http://nlp.stanford.edu/data/glove.6B.zip
* !unzip glove.6B.zip

In [27]:
GLOVE_EMB = '/kaggle/working/glove.6B.300d.txt'
EMBEDDING_DIM = 300
LR = 1e-3
BATCH_SIZE = 1024
EPOCHS = 15
MODEL_PATH = '.../output/kaggle/working/best_model.hdf5'

In [28]:
embeddings_index = {}

f = open(GLOVE_EMB)
for line in f:
    values = line.split()
    word = value = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/glove.6B.300d.txt'

In [29]:
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

NameError: name 'vocab_size' is not defined

In [30]:
embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                            EMBEDDING_DIM,
                                            weights=[embedding_matrix],
                                            input_length = 30,
                                            trainable=False)

NameError: name 'vocab_size' is not defined

### Modeling

In [31]:
sequence_input = Input(shape=(30,),dtype = 'int32')

embedding_sequences = embedding_layer(sequence_input)
net = SpatialDropout1D(0.2)(embedding_sequences)
net = Conv1D(64,5,activation = 'relu')(net)
net = Bidirectional(LSTM(64,dropout=0.2, recurrent_dropout=0.2))(net)
net = Dense(512,activation = 'relu')(net)
net = Dense(1,activation = 'sigmoid')(net)

outputs = net
model = tf.keras.Model(sequence_input,outputs)

NameError: name 'embedding_layer' is not defined

In [32]:
model.compile(optimizer = Adam(learning_rate = LR),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

NameError: name 'model' is not defined

In [33]:
ReduceLROnPlateau = ReduceLROnPlateau(factor=0.1,
                                     min_lr = 0.01,
                                     monitor = 'val_loss',
                                     verbose = 1)

In [34]:
history = model.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=20,
                    validation_data=(x_test, y_test),
                    callbacks=[ReduceLROnPlateau])

NameError: name 'model' is not defined

In [35]:

s, (at, al) = plt.subplots(2,1,figsize=(20,15))

at.plot(history.history['accuracy'], c= 'b')
at.plot(history.history['val_accuracy'], c='r')
at.set_title('model accuracy')
at.set_ylabel('accuracy')
at.set_xlabel('epoch')
at.legend(['LSTM_train', 'LSTM_val'], loc='upper left')


al.plot(history.history['loss'], c='m')
al.plot(history.history['val_loss'], c='c')
al.set_title('model loss')
al.set_ylabel('loss')
al.set_xlabel('epoch')
al.legend(['train', 'val'], loc = 'upper left')

NameError: name 'plt' is not defined

* epochs을 7~1로 수정하는 것을 권장